In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
data = pd.read_csv('C:/Users/plomo/kaggle/archive/movies_metadata.csv')

C:\Users\plomo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
data = data.head(20000)

In [4]:
data['overview'].isnull().sum()

135

In [5]:
data['overview'] = data['overview'].fillna('')

In [6]:
tfidf = TfidfVectorizer(stop_words='english')
#overview에 대해 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(data['overview'])
print(tfidf_matrix.shape)
#20000개의 영화를 표현하기 위해 47487개의 단어가 사용

(20000, 47487)


In [7]:
#코사인 유사도
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [8]:
#영화 타이틀과 인덱스를 가진 테이블을 만든다
#영화 타이틀 입력하면 인덱스를 리턴하기 위해
indices = pd.Series(data.index, index=data['title']).drop_duplicates()
print(indices.head())

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64


In [9]:
idx = indices['Father of the Bride Part II']
print(idx)

4


In [10]:
def get_recommendations(title, cosine_sim=cosine_sim):
    #영화의 타이틀로부터 해당되는 인덱스를 받아옴
    idx = indices[title]
    #모든 영화에 대해서 해당 영화와의 유사도 구함
    sim_scores = list(enumerate(cosine_sim[idx]))
    #유사도에 따라 영화들을 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    #가장 유사한 10개
    sim_scores = sim_scores[1:11]
    #가장 유사란 10개 영화의 인덱스 받아옴
    movie_indices = [i[0] for i in sim_scores]
    
    #가장 유사한 10개 영화 제목 리턴
    return data['title'].iloc[movie_indices]

In [11]:
get_recommendations('The Dark Knight Rises')

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object